# 국토교통부 '아파트 실거래가' 오픈 API 수집 방법
$$ $$
* 국토교통부는 2014년부터 부동산 거래금액을 오픈 API로 제공하고 있습니다.
  - [국토교통부 아파트 실거래가 사이트](https://www.data.go.kr/dataset/3050988/openapi.do)
  - 아파트, 단독/다가구, 오피스텔, 토지 매매 등 총 10가지 항목이 포함되어 있습니다. 
$$ $$
* [공공데이터 포털](http://www.data.go.kr)에서 인증키를 발급받으시면 됩니다.
  - 인증키 발급방법은 ['공공데이터포털 OpenAPI 인증키 발급방법'](http://nbviewer.jupyter.org/format/slides/gist/KevinSHNa/89aabf84f7e8eae51a334c87a624e31f#/)을 참고하시기 바랍니다.

## URL 조립 방법 안내
$$ $$
* 공통으로 사용되는 부분(main)과 세부항목(아파트매매 실거래가), 조회년월, 지역코드 및 인증키로 조립합니다. 
  - 아래 url은 아파트매매 실거래가에서 사용되는 예시입니다. 
  - http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?ServiceKey=인증키

In [1]:
# 필요한 라이브러리를 불러옵니다. 
library(httr)
library(rvest)
library(dplyr)
library(stringr)

Loading required package: xml2

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# url 요소를 설정합니다. 
# 공통부분(main)
main <- "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/"

# 아파트매매 실거래가 
sub1 <- "RTMSOBJSvc/getRTMSDataSvcAptTrade"

# 인증키를 입력합니다. 
api_key <- "발급받은 인증키를 이곳에 입력하세요"

In [3]:
# 실제 인증키를 할당합니다. 
api_key <- "e5rYGavudWWkTkvxiFH7tJlfSSDX9EczjpD1k2zvLcrt5qJjIAKkX5aNRYj8wqxapIYlJZ0XwQ4ZY%2BKNFn5TEQ%3D%3D"

In [4]:
# url을 조립합니다. 
# 조회년월은 "201709"로 하고, 지역코드는 "11110"(서울특별시 종로구)로 설정합니다. 
url <- paste0(main, 
              sub1, 
              paste0("?ServiceKey=", api_key),
              paste0("&LAWD_CD=", "11110"),
              paste0("&DEAL_YMD=", "201709"))

In [5]:
# url을 확인합니다.
url

[1] "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?ServiceKey=e5rYGavudWWkTkvxiFH7tJlfSSDX9EczjpD1k2zvLcrt5qJjIAKkX5aNRYj8wqxapIYlJZ0XwQ4ZY%2BKNFn5TEQ%3D%3D&LAWD_CD=11110&DEAL_YMD=201709"

In [6]:
# 조립한 url로 요청합니다. 
resp <- GET(url)
resp$status_code

[1] 200

In [7]:
# resp 객체의 content를 텍스트로 추출하여 구조를 파악합니다. 
# 만약 encoding이 'EUC-KR'로 되어 있으면, 
# content() 인자로 endcoding="EUC-KR"을 추가합니다. 
xmlObj <- content(resp, as="text")
cat(xmlObj)

# response > body > items 하위 노드로 item이 반복되고 있습니다. 

No encoding supplied: defaulting to UTF-8.


{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL SERVICE."},"body":{"items":{"item":[{"거래금액":"    87,000","건축년도":2000,"년":2017,"법정동":" 청운동","아파트":"청운현대","월":9,"일":"11~20","전용면적":103.67,"지번":"56-45","지역코드":11110,"층":2},{"거래금액":"    86,500","건축년도":2008,"년":2017,"법정동":" 사직동","아파트":"광화문풍림스페이스본(9-0)","월":9,"일":"1~10","전용면적":94.51,"지번":9,"지역코드":11110,"층":8},{"거래금액":"    64,800","건축년도":1983,"년":2017,"법정동":" 당주동","아파트":"세종","월":9,"일":"11~20","전용면적":104.5,"지번":100,"지역코드":11110,"층":10},{"거래금액":"   116,700","건축년도":2004,"년":2017,"법정동":" 내수동","아파트":"킹스매너","월":9,"일":"1~10","전용면적":201.82,"지번":"110-15","지역코드":11110,"층":1},{"거래금액":"   117,500","건축년도":2004,"년":2017,"법정동":" 내수동","아파트":"경희궁의아침4단지","월":9,"일":"1~10","전용면적":124.17,"지번":73,"지역코드":11110,"층":2},{"거래금액":"    37,500","건축년도":2004,"년":2017,"법정동":" 이화동","아파트":"(9-1)","월":9,"일":"1~10","전용면적":84.42,"지번":"9-1","지역코드":11110,"층":5},{"거래금액":"    18,300","건축년도":2014,"년":2017,"법정동":" 연건동","아파트":"이화에수풀","월":9,"일":"1~10","전용면적":16.98,"지번":"195-10"

In [8]:
# 이제 XML을 불러옵니다. 
xml <- read_xml(url)
xml

{xml_document}
<response>
[1] <header>\n  <resultCode>00</resultCode>\n  <resultMsg>NORMAL SERVICE.</re ...
[2] <body>\n  <items>\n    <item>\n      <거래금액>    87,000</거래금액>\n      <건축년도 ...

In [9]:
# 조회건수(totalCount)를 확인합니다. 
xml %>% xml_nodes("body totalCount") %>% xml_text()

[1] "30"

In [10]:
# 거래 리스트를 수집합니다. 
items <- xml %>% xml_nodes("item")
items

{xml_nodeset (30)}
 [1] <item>\n  <거래금액>    87,000</거래금액>\n  <건축년도>2000</건축년도>\n  <년>2017</년>\n  ...
 [2] <item>\n  <거래금액>    86,500</거래금액>\n  <건축년도>2008</건축년도>\n  <년>2017</년>\n  ...
 [3] <item>\n  <거래금액>    64,800</거래금액>\n  <건축년도>1983</건축년도>\n  <년>2017</년>\n  ...
 [4] <item>\n  <거래금액>   116,700</거래금액>\n  <건축년도>2004</건축년도>\n  <년>2017</년>\n  ...
 [5] <item>\n  <거래금액>   117,500</거래금액>\n  <건축년도>2004</건축년도>\n  <년>2017</년>\n  ...
 [6] <item>\n  <거래금액>    37,500</거래금액>\n  <건축년도>2004</건축년도>\n  <년>2017</년>\n  ...
 [7] <item>\n  <거래금액>    18,300</거래금액>\n  <건축년도>2014</건축년도>\n  <년>2017</년>\n  ...
 [8] <item>\n  <거래금액>    35,800</거래금액>\n  <건축년도>1993</건축년도>\n  <년>2017</년>\n  ...
 [9] <item>\n  <거래금액>    37,600</거래금액>\n  <건축년도>1992</건축년도>\n  <년>2017</년>\n  ...
[10] <item>\n  <거래금액>    36,000</거래금액>\n  <건축년도>1993</건축년도>\n  <년>2017</년>\n  ...
[11] <item>\n  <거래금액>    37,900</거래금액>\n  <건축년도>1992</건축년도>\n  <년>2017</년>\n  ...
[12] <item>\n  <거래금액>    50,900</거래금액>\n  <건축년도>1993</건축년도>\n  <년>2017</년>\n  .

In [11]:
# 아파트만 문자열 벡터로 추출해보겠습니다. 
items %>% xml_nodes("아파트") %>% xml_text()

[1] "청운현대"                  "광화문풍림스페이스본(9-0)"
 [3] "세종"                      "킹스매너"                 
 [5] "경희궁의아침4단지"         "(9-1)"                    
 [7] "이화에수풀"                "창신쌍용2"                
 [9] "창신쌍용1"                 "창신쌍용2"                
[11] "창신쌍용1"                 "창신쌍용2"                
[13] "창신쌍용1"                 "창신쌍용2"                
[15] "종로청계힐스테이트"        "종로센트레빌"             
[17] "종로센트레빌"              "종로센트레빌"             
[19] "종로센트레빌"              "종로센트레빌"             
[21] "인왕산2차아이파크"         "인왕산2차아이파크"        
[23] "현대"                      "인왕산아이파크"           
[25] "인왕산아이파크"            "인왕산아이파크"           
[27] "인왕산아이파크"            "현대"                     
[29] "현대"                      "인왕산아이파크"

In [12]:
# 위와 같은 방법으로 텍스트를 추출해주는 나만의 함수를 하나 만듭니다. 
getXmlText <- function(x, var) {
    result <- x %>% xml_node(var) %>% xml_text()
    return(result)
}

In [13]:
# 나만의 함수를 테스트합니다. 
itemNm <- getXmlText(items, "아파트")
itemNm

[1] "청운현대"                  "광화문풍림스페이스본(9-0)"
 [3] "세종"                      "킹스매너"                 
 [5] "경희궁의아침4단지"         "(9-1)"                    
 [7] "이화에수풀"                "창신쌍용2"                
 [9] "창신쌍용1"                 "창신쌍용2"                
[11] "창신쌍용1"                 "창신쌍용2"                
[13] "창신쌍용1"                 "창신쌍용2"                
[15] "종로청계힐스테이트"        "종로센트레빌"             
[17] "종로센트레빌"              "종로센트레빌"             
[19] "종로센트레빌"              "종로센트레빌"             
[21] "인왕산2차아이파크"         "인왕산2차아이파크"        
[23] "현대"                      "인왕산아이파크"           
[25] "인왕산아이파크"            "인왕산아이파크"           
[27] "인왕산아이파크"            "현대"                     
[29] "현대"                      "인왕산아이파크"

In [14]:
# 이제 필요한 필요한 컬럼들로 구성된 데이터프레임을 생성해보겠습니다. 
itemList <- data.frame(price = getXmlText(items, "거래금액"),
                       built = getXmlText(items, "건축년도"),
                       year = getXmlText(items, "년"),
                       dong = getXmlText(items, "법정동"),
                       aptNm = getXmlText(items, "아파트"),
                       month = getXmlText(items, "월"),
                       date = getXmlText(items, "일"),
                       size = getXmlText(items, "전용면적"),
                       jibun = getXmlText(items, "지번"),
                       area = getXmlText(items, "지역코드"),
                       floor = getXmlText(items, "층"))

head(itemList)

price,built,year,dong,aptNm,month,date,size,jibun,area,floor
"87,000",2000,2017,청운동,청운현대,9,11~20,103.67,56-45,11110,2
"86,500",2008,2017,사직동,광화문풍림스페이스본(9-0),9,1~10,94.51,9,11110,8
"64,800",1983,2017,당주동,세종,9,11~20,104.5,100,11110,10
"116,700",2004,2017,내수동,킹스매너,9,1~10,201.82,110-15,11110,1
"117,500",2004,2017,내수동,경희궁의아침4단지,9,1~10,124.17,73,11110,2
"37,500",2004,2017,이화동,(9-1),9,1~10,84.42,9-1,11110,5


## 기준년월과 지역코드로 수집하는 나만의 함수를 생성합니다.
$$ $$
- 입력할 인자는 조회년월과 지역코드 2가지입니다. 
- 지역코드는 "실거래 데이터 OpenAPI 활용가이드 아파트 매매 V2.docx" 부록에서 발췌하였습니다. 
  * area_code.txt로 제 깃헙에 업로드하였습니다. 
  * "https://raw.githubusercontent.com/KevinSHNa/MLR/master/area_code.txt"
- 아래 함수가 실행되면 기준년월, 지역명 및 거래건수가 출력됩니다. 

In [15]:
# 지역코드와 지역명을 불러옵니다. 
areaCode <- read.table("https://raw.githubusercontent.com/KevinSHNa/MLR/master/area_code.txt", header=T)
areaCode$코드 <- as.character(areaCode$코드)

In [16]:
# 나만의 함수를 생성합니다. 
getAptPrice <- function(month, areaCd) {
  
  # url을 조립합니다. 
  url <- paste0(main, 
                sub1, 
                paste0("?ServiceKey=", api_key),
                paste0("&LAWD_CD=", areaCd),
                paste0("&DEAL_YMD=", month))
  #print(url)
  
  # 조립한 url로 요청합니다. 
  resp <- GET(url)
  
  # 이제 XML을 불러옵니다. 
  xml <- read_xml(url)
  
  # 조회건수(totalCount)를 확인합니다. 
  cnt <- xml %>% xml_nodes("body totalCount") %>% xml_text()
  
  # 지역명 
  areaNm <- areaCode$지역[areaCode$코드==areaCd]
  
  cat("기준월:", month, "/", "지역명:", paste0(areaNm, " (", areaCd ,")") , "/", "조회건수:", cnt, "건", "\n")
  
  # 품목 리스트를 수집합니다. 
  items <- xml %>% xml_nodes("item")
  
  # 이제 필요한 필요한 컬럼들로 구성된 데이터프레임을 생성해보겠습니다. 
  itemList <- data.frame(month = month, 
                         date = getXmlText(items, "일"),
                         area = getXmlText(items, "지역코드"),
                         aptNm = getXmlText(items, "아파트"),
                         size = getXmlText(items, "전용면적"),
                         floor = getXmlText(items, "층"),
                         price = getXmlText(items, "거래금액"),
                         built = getXmlText(items, "건축년도"),
                         dong = getXmlText(items, "법정동"),
                         jibun = getXmlText(items, "지번"))
  
  # 전용면적, 층, 거래금액을 숫자형으로 변환하기 
  itemList$size <- as.numeric(itemList$size)
  itemList$floor <- as.numeric(itemList$floor)
  itemList$price <- as.numeric(gsub(",", "", itemList$price))
  
  # 법정동, 아파트명, 전용면적, 층 기준으로 정렬하기
  itemList <- itemList[order(itemList$dong, itemList$aptNm, itemList$size, itemList$floor),]
  
  return(itemList)
}

In [17]:
# 조회년월과 지역코드를 순환하며 수집합니다. 
# 조회기간은 2017년 7~9월로 설정합니다. 
months <- c("201707","201708","201709")

# 서울특별시 25개 자치구에 대해서 수집합니다. 
areaCds <- areaCode$코드[grep("서울", areaCode$지역)]

# 최종 데이터프레임 초기화합니다. 
aptPrice <- data.frame()

# 위에서 설정한 조회기간과 지역코드를 기준으로 데이터를 수집합니다. 
for (month in months) {
  for (areaCd in areaCds) {
    df <- getAptPrice(month, areaCd)
    aptPrice <- rbind(aptPrice, df)
  }
}

기준월: 201707 / 지역명: 서울특별시 종로구 (11110) / 조회건수: 99 건 
기준월: 201707 / 지역명: 서울특별시 중구 (11140) / 조회건수: 151 건 
기준월: 201707 / 지역명: 서울특별시 용산구 (11170) / 조회건수: 309 건 
기준월: 201707 / 지역명: 서울특별시 성동구 (11200) / 조회건수: 616 건 
기준월: 201707 / 지역명: 서울특별시 광진구 (11215) / 조회건수: 320 건 
기준월: 201707 / 지역명: 서울특별시 동대문구 (11230) / 조회건수: 533 건 
기준월: 201707 / 지역명: 서울특별시 중랑구 (11260) / 조회건수: 435 건 
기준월: 201707 / 지역명: 서울특별시 성북구 (11290) / 조회건수: 702 건 
기준월: 201707 / 지역명: 서울특별시 강북구 (11305) / 조회건수: 277 건 
기준월: 201707 / 지역명: 서울특별시 도봉구 (11320) / 조회건수: 679 건 
기준월: 201707 / 지역명: 서울특별시 노원구 (11350) / 조회건수: 1898 건 
기준월: 201707 / 지역명: 서울특별시 은평구 (11380) / 조회건수: 362 건 
기준월: 201707 / 지역명: 서울특별시 서대문구 (11410) / 조회건수: 376 건 
기준월: 201707 / 지역명: 서울특별시 마포구 (11440) / 조회건수: 472 건 
기준월: 201707 / 지역명: 서울특별시 양천구 (11470) / 조회건수: 688 건 
기준월: 201707 / 지역명: 서울특별시 강서구 (11500) / 조회건수: 990 건 
기준월: 201707 / 지역명: 서울특별시 구로구 (11530) / 조회건수: 624 건 
기준월: 201707 / 지역명: 서울특별시 금천구 (11545) / 조회건수: 197 건 
기준월: 201707 / 지역명: 서울특별시 영등포구 (11560) / 조회건수: 520 건 
기준월: 20170

# End of Document